# Unpaired Style Transfer using CycleGAN
[CycleGAN](https://arxiv.org/pdf/1703.10593.pdf) improves upon paired style transfer architecture by relaxing the constraint on input and output images. CycleGAN explores the unpaired style transfer paradigm where the model actually tries to learn the stylistic differences between source and target domains without explicit pairing between input and output images. Zhu and Park et al. describe this unpaired style transfer similar to our ability of imagining how a Van Gogh or Monet would have painted a particular scene (without having actually seen a side by side example). Quoting from the paper  itself,

> Instead, we have knowledge of the set of Monet paintings and of the set of landscape photographs. We can reason about the stylistic differences between these two sets, and thereby imagine what a scene might look like if we were to “translate” it from one set into the other.


This provides a nice advantage as well as opens additional use cases where exact pairing of source and target domains is either not available or we do not have enough training examples.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/PacktPublishing/Hands-On-Generative-AI-with-Python-and-TensorFlow-2/blob/master/Chapter_7/cycleGAN/cycleGAN.ipynb)

## Load Libraries

In [1]:
from tensorflow.keras.layers import Input, Concatenate
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np

## Load Utilities

In [ ]:
from gan_utils import downsample_block, upsample_block, discriminator_block
from data_utils import plot_sample_images, batch_generator, get_samples

## Set Configs

In [7]:
params = {'legend.fontsize': 'x-large',
          'figure.figsize': (8,8),
         'axes.labelsize': 'x-large',
         'axes.titlesize':'x-large',
         'xtick.labelsize':'x-large',
         'ytick.labelsize':'x-large'}

plt.rcParams.update(params)

In [8]:
IMG_WIDTH = 128
IMG_HEIGHT = 128
dataset_name = 'apple2orange'

DOWNLOAD_URL = 'https://efrosgans.eecs.berkeley.edu/cyclegan/datasets/{}.zip'.format(dataset_name)

## U-Net Generator

In [9]:
def build_generator(img_shape, channels=3, num_filters=32):
    # Image input
    input_layer = Input(shape=img_shape)

    # Downsampling
    down_sample_1 = downsample_block(input_layer, num_filters)
    down_sample_2 = downsample_block(down_sample_1, num_filters*2)
    down_sample_3 = downsample_block(down_sample_2,num_filters*4)
    down_sample_4 = downsample_block(down_sample_3,num_filters*8)

    # Upsampling
    upsample_1 = upsample_block(down_sample_4, down_sample_3, num_filters*4)
    upsample_2 = upsample_block(upsample_1, down_sample_2, num_filters*2)
    upsample_3 = upsample_block(upsample_2, down_sample_1, num_filters)

    upsample_4 = UpSampling2D(size=2)(upsample_3)
    output_img = Conv2D(channels, 
                        kernel_size=4, 
                        strides=1, 
                        padding='same', 
                        activation='tanh')(upsample_4)

    return Model(input_layer, output_img)

## Discriminator

In [10]:
def build_discriminator(img_shape,num_filters=64):
    input_layer = Input(shape=img_shape)

    disc_block_1 = discriminator_block(input_layer, 
                                       num_filters, 
                                       instance_normalization=False)
    disc_block_2 = discriminator_block(disc_block_1, num_filters*2)
    disc_block_3 = discriminator_block(disc_block_2, num_filters*4)
    disc_block_4 = discriminator_block(disc_block_3, num_filters*8)

    output = Conv2D(1, kernel_size=4, strides=1, padding='same')(disc_block_4)

    return Model(input_layer, output)

## GAN Configuration

In [11]:
generator_filters = 32
discriminator_filters = 64

# input shape
channels = 3
input_shape = (IMG_HEIGHT, IMG_WIDTH, channels)

# Loss weights
lambda_cycle = 10.0            
lambda_identity = 0.1 * lambda_cycle

optimizer = Adam(0.0002, 0.5)

In [12]:
# prepare patch size for our setup
patch = int(IMG_HEIGHT / 2**4)
patch_gan_shape = (patch, patch, 1)
print("Patch Shape={}".format(patch_gan_shape))

Patch Shape=(8, 8, 1)


## Get Discriminators

In [13]:
disc_A = build_discriminator(input_shape,discriminator_filters)
disc_A.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

disc_B = build_discriminator(input_shape,discriminator_filters)
disc_B.compile(loss='mse',
    optimizer=optimizer,
    metrics=['accuracy'])

## Get Generators and GAN Model Objects

In [14]:
gen_AB = build_generator(input_shape, channels, generator_filters)
gen_BA = build_generator(input_shape, channels, generator_filters)

In [15]:
img_A = Input(shape=input_shape)
img_B = Input(shape=input_shape)

# generate fake samples from both generators
fake_B = gen_AB(img_A)
fake_A = gen_BA(img_B)

# reconstruct orginal samples from both generators
reconstruct_A = gen_BA(fake_B)
reconstruct_B = gen_AB(fake_A)

# generate identity samples
identity_A = gen_BA(img_A)
identity_B = gen_AB(img_B)

# disable discriminator training
disc_A.trainable = False
disc_B.trainable = False

# use discriminator to classify real vs fake
output_A = disc_A(fake_A)
output_B = disc_B(fake_B)

# Combined model trains generators to fool discriminators
gan = Model(inputs=[img_A, img_B],
            outputs=[output_A, output_B,
                     reconstruct_A, reconstruct_B,
                     identity_A, identity_B ])
gan.compile(loss=['mse', 'mse','mae', 'mae','mae', 'mae'],
            loss_weights=[1, 1,
                          lambda_cycle, lambda_cycle,
                          lambda_identity, lambda_identity ],
            optimizer=optimizer)

## Custom Training Loop

In [16]:
def train(gen_AB, 
          gen_BA, 
          disc_A, 
          disc_B, 
          gan, 
          patch_gan_shape, 
          epochs, 
          path='./content/{}'.format(dataset_name) ,
          batch_size=1, 
          sample_interval=50):

    # Adversarial loss ground truths
    real_y = np.ones((batch_size,) + patch_gan_shape)
    fake_y = np.zeros((batch_size,) + patch_gan_shape)

    imgs = batch_generator(path, batch_size, image_res=[IMG_HEIGHT, IMG_WIDTH])
    print(imgs)

    for epoch in range(epochs):
        print("Epoch={}".format(epoch))
        for idx, (imgs_A, imgs_B) in enumerate(batch_generator(path,
                                                               batch_size,
                                                               image_res=[IMG_HEIGHT, IMG_WIDTH])):

            # train discriminators

            # generate fake samples from both generators
            fake_B = gen_AB.predict(imgs_A)
            fake_A = gen_BA.predict(imgs_B)

            # Train the discriminators (original images = real / translated = Fake)
            disc_A_loss_real = disc_A.train_on_batch(imgs_A, real_y)
            disc_A_loss_fake = disc_A.train_on_batch(fake_A, fake_y)
            disc_A_loss = 0.5 * np.add(disc_A_loss_real, disc_A_loss_fake)

            disc_B_loss_real = disc_B.train_on_batch(imgs_B, real_y)
            disc_B_loss_fake = disc_B.train_on_batch(fake_B, fake_y)
            disc_B_loss = 0.5 * np.add(disc_B_loss_real, disc_B_loss_fake)

            # Total disciminator loss
            discriminator_loss = 0.5 * np.add(disc_A_loss, disc_B_loss)


            # train generator
            gen_loss = gan.train_on_batch([imgs_A, imgs_B],
                                          [
                                           real_y, real_y,
                                           imgs_A, imgs_B,
                                           imgs_A, imgs_B
                                           ]
                                          )

            # training updates every 50 iterations
            if idx % 50 == 0:
              print ("[Epoch {}/{}] [Discriminator loss: {}, accuracy: {}][Generator loss: {}, Adversarial Loss: {}, Reconstruction Loss: {}, Identity Loss: {}]".format(idx, 
                                                  epoch,
                                                  discriminator_loss[0], 
                                                  100*discriminator_loss[1],
                                                  gen_loss[0],
                                                  np.mean(gen_loss[1:3]),
                                                  np.mean(gen_loss[3:5]),
                                                  np.mean(gen_loss[5:6])))
              
            # Plot and Save progress every few iterations
            if idx % sample_interval == 0:
              plot_sample_images(gen_AB,
                                 gen_BA,
                                 path=path,
                                 epoch=epoch,
                                 batch_num=idx,
                                 output_dir='images')

## Download Dataset

In [17]:
import os
path = os.getcwd()
path = os.path.join(path, 'content')

tf.keras.utils.get_file('{}.tar.gz'.format(dataset_name),
                         origin=DOWNLOAD_URL,
                         cache_subdir=path,
                         extract=True)

78471168/78456409 [==============================] - 9s 0us/step


'D:\\Home\\Project\\MachineLearning\\GAN\\gan_utils\\content\\apple2orange.tar.gz'

## Training Begins!

In [ ]:
train(gen_AB, 
      gen_BA, 
      disc_A, 
      disc_B, 
      gan, 
      patch_gan_shape, 
      epochs=200, 
      batch_size=1, 
      sample_interval=200)

# End of the program